In [2]:
import torch
from torch import nn
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR, MultiStepLR
import torchvision
from torchvision import transforms, utils, datasets

import os
import sys
if '/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2
import h5py
import PIL
import numpy as np
import random
import logging
import gdown
from zipfile import ZipFile
import matplotlib.pyplot as plt

%matplotlib inline

## Parameters

In [3]:
RATIO_VALIDATION_SPLIT = 0.1
BATCH_SIZE = 8
NUM_WROKERS = 4
INPUT_IMG_SIZE = (101, 101)
EPOCH = 50               # train the training data n times, to save time, we just train 1 epoch
LR = 0.001               # learning rate
DATASET_ROOT = './3-class_behavior-reflex'
MODELS_ROOT = './models'
CLASSES = np.loadtxt('class_id.txt', str, delimiter='\n')

## Download dataset

In [4]:
train_dataset_url = 'https://drive.google.com/uc?id=10wJUsO2Eo6aOK-ay-UlBBlf1_Ba1LdTY'
train_dataset_name = '3-class_behavior-reflex'
if not os.path.isdir(train_dataset_name):
    gdown.download(train_dataset_url, output=train_dataset_name + '.zip', quiet=False)
    zip1 = ZipFile(train_dataset_name + '.zip')
    zip1.extractall(train_dataset_name)
    zip1.close()

print 'Finished downloading training dataset.'

Finished downloading training dataset.


## Download pre-trained model

In [5]:
pretrain_model_url = 'https://drive.google.com/uc?id=1aSU1mg0qiFsgBw1p2-hd2bgv2WtQh2m_'
pretrain_model_name = 'trailnet_3class_real_vr_mix_color.h5'
if not os.path.isfile(os.path.join(MODELS_ROOT, pretrain_model_name)):
    if not os.path.isdir(MODELS_ROOT):
        os.mkdir(MODELS_ROOT)
    gdown.download(pretrain_model_url, output=os.path.join(MODELS_ROOT, pretrain_model_name), quiet=False)
    
print 'Finished downloading pre-trained model.'  

Finished downloading pre-trained model.


## Useful functions: 
dataset visualization and image loading function

In [6]:
def vis_img(batch_data):   
    # show images
    imgs = torchvision.utils.make_grid(batch_data)
    imgs = imgs / 2 + 0.5     # unnormalize
    npimgs = imgs.numpy()
    plt.rcParams['figure.figsize'] = [12, 5]
    plt.imshow(np.transpose(npimgs, (1, 2, 0)))

In [7]:
def load_images_from_folder(folder, classes):
    img_paths = []
    labels = []
    for class_id, class_name in enumerate(classes):
        class_folder = os.path.join(folder, class_name)
        for filename in os.listdir(class_folder):
            filename.lower().endswith(('.png', '.jpg', '.jpeg'))
            img_paths.append(os.path.join(class_folder, filename))
            labels.append(class_id)
    return img_paths, labels

## Custom pytorch dataset class for Trailnet (load dataset from txt)

In [21]:
class TrailnetDataset(Dataset):
    def __init__(self, datalist_filename):
        classes = CLASSES
        
        self.img_list = []
        self.label_list = []
        
        txt = np.loadtxt(datalist_filename, str, delimiter='\n')
        for line in txt:
            p, l  = line.split()
            self.img_list.append(p)
            self.label_list.append(l)
        
        print '********** Dataset Info start **********\n'
        print 'Source: ', datalist_filename
        print 'Output classes: ', classes
        print 'Amount of images: ', len(txt)
        print '\n*********** Dataset Info end ***********\n'
        
        self.data_transform = transforms.Compose([ 
                                transforms.Resize(INPUT_IMG_SIZE), \
                                transforms.ToTensor(), \
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], \
                                                     std=[1, 1, 1]), \
                                ])
        
    def __len__(self):
        return len(self.img_list)
    
    def collect_folders_from_dataset(self, dataset_root, classes):
        # Implement by BFS
        search_list = [dataset_root, ]
        dataset_folders = [] 
        while len(search_list) != 0:
            root = search_list.pop(0)
            if set(os.listdir(root)) == set(classes):
                dataset_folders.append(root)
            else:
                for folder in os.listdir(root):
                    path = os.path.join(root, folder)
                    if os.path.isdir(path):
                        search_list.append(path)
        return dataset_folders

    def __getitem__(self, index):
        'Generates one sample of data'
        # print self.img_list[index]
        # Select sample, then load data and get label
        path = self.img_list[index]
        img_raw = self.default_loader(path)
        x = self.data_transform(img_raw)
        y = torch.tensor(self.label_list[index])
        return x, y, path
    
    def pil_loader(self, path):
        with open(path, 'rb') as f:
            with PIL.Image.open(f) as img:
                return img.convert('RGB')

    def accimage_loader(self, path):
        try:
            return accimage.Image(path)
        except IOError:
            # Potentially a decoding problem, fall back to PIL.Image
            return pil_loader(path)

    def default_loader(self, path):
        if torchvision.get_image_backend() == 'accimage':
            return self.accimage_loader(path)
        else:
            return self.pil_loader(path)

## Train / validation dataset declaration 

In [23]:
train_dataset = TrailnetDataset(datalist_filename='train_dataset.txt')
val_dataset = TrailnetDataset(datalist_filename='validation_dataset.txt')

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WROKERS, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WROKERS, shuffle=False)

********** Dataset Info start **********

Source:  train_dataset.txt
Output classes:  ['L' 'S' 'R']
Amount of images:  26922

*********** Dataset Info end ***********

********** Dataset Info start **********

Source:  validation_dataset.txt
Output classes:  ['L' 'S' 'R']
Amount of images:  2992

*********** Dataset Info end ***********



## Training and Testing function

In [11]:
def train(loader, model, criterion, optimizer, device, debug_steps=100, epoch=-1):
    model.train(True)
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    for i, data in enumerate(loader):
        images, labels, _path = data
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)  # TODO CHANGE BOXES
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i and i % debug_steps == 0:
            avg_loss = loss / debug_steps
            # logging.info("Epoch: {}, Step: {}, Average Loss: {:.4f}".format(epoch, i, avg_loss))
            print "Epoch: {}, Step: {}, Average Loss: {:.6f}".format(epoch, i, avg_loss)
            running_loss = 0.0

def test(loader, model, criterion, device):
    model.eval()
    running_loss = 0.0
    num = 0
    for _, data in enumerate(loader):
        images, labels, _path = data
        images = images.to(device)
        labels = labels.to(device)
        num += 1

        with torch.no_grad():
            outputs = model(images)
            loss = criterion(outputs, labels)

        running_loss += loss.item()
    return running_loss / num

## Trailnet 3 class output model

In [12]:
class TrailNet3Class(nn.Module):
    def __init__(self):
        super(TrailNet3Class, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 32, 4)
        self.pool1 = nn.MaxPool2d((2, 2), stride=2)
        self.conv2 = nn.Conv2d(32, 32, 4)
        self.pool2 = nn.MaxPool2d((2, 2), stride=2)
        self.conv3 = nn.Conv2d(32, 32, 4)
        self.pool3 = nn.MaxPool2d((2, 2), stride=2)
        self.conv4 = nn.Conv2d(32, 32, 4, padding=(2, 2))
        self.pool4 = nn.MaxPool2d((2, 2), stride=2)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(800, 200)
        self.fc2 = nn.Linear(200, 3)

    def forward(self, x):
        x = self.pool1(self.conv1(x))
        x = self.pool2(self.conv2(x))
        x = self.pool3(self.conv3(x)) 
        x = self.pool4(self.conv4(x))    
        # print 'x size: ', x.size()   
        x = x.view(-1, self.num_flat_features(x))
        # print 'x size: ', x.size()   
        x = self.fc1(x)
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [13]:
def load_pretrained_model(model, filename='trailnet_3class_real_vr_mix_color.h5', use_cuda=True):
    if filename.lower().endswith('.h5'):
        state_dict = h5py.File(os.path.join(MODELS_ROOT,filename), 'r')
        model.load_state_dict({l : torch.from_numpy(np.array(v)).view_as(p) \
                     for k, v in state_dict.items() \
                     for l, p in model.named_parameters() if k in l})
    elif filename.lower().endswith('.pth'):
        state_dict = torch.load(os.path.join(MODELS_ROOT, filename))
        model.load_state_dict(state_dict)
        
    if torch.cuda.is_available() and use_cuda:
        cudnn.benchmark = True
        model.cuda()

In [14]:
model = TrailNet3Class()
load_pretrained_model(model)

## Training process

In [15]:
last_epoch = -1
validation_epochs = 5

params = model.parameters()

checkpoint_folder = './models'
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params, lr=1e-3, momentum=0.9, weight_decay=5e-4)
scheduler = CosineAnnealingLR(optimizer, 120, last_epoch=last_epoch)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for epoch in range(last_epoch + 1, EPOCH):
    scheduler.step()
    train(train_loader, model, criterion, optimizer, \
          device=DEVICE, debug_steps=100, epoch=epoch)

    if epoch % validation_epochs == 0 or epoch == EPOCH - 1:
        val_loss = test(val_loader, model, criterion, DEVICE)
        # logging.info("Epoch: {}, ".format(epoch) + \
        #            "Validation Loss: {:.4f}".format(val_loss))
        print "Epoch: {}, Validation Loss: {:.6f}".format(epoch, val_loss)
        model_path = os.path.join(checkpoint_folder, \
                                  "{}_epoch{}_loss{:.5f}.pth".format('trailnet_3class', epoch, val_loss))
        torch.save(model.state_dict(), model_path)
        # logging.info("Saved model {}".format(model_path))


/usr/local/lib/python2.7/dist-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch: 0, Step: 100, Average Loss: 0.000400
Epoch: 0, Step: 200, Average Loss: 0.000006
Epoch: 0, Step: 300, Average Loss: 0.000002
Epoch: 0, Step: 400, Average Loss: 0.000006
Epoch: 0, Step: 500, Average Loss: 0.000399
Epoch: 0, Step: 600, Average Loss: 0.000002
Epoch: 0, Step: 700, Average Loss: 0.000016
Epoch: 0, Step: 800, Average Loss: 0.000019
Epoch: 0, Step: 900, Average Loss: 0.000006
Epoch: 0, Step: 1000, Average Loss: 0.000025
Epoch: 0, Step: 1100, Average Loss: 0.000004
Epoch: 0, Step: 1200, Average Loss: 0.000000
Epoch: 0, Step: 1300, Average Loss: 0.000001
Epoch: 0, Step: 1400, Average Loss: 0.000007
Epoch: 0, Step: 1500, Average Loss: 0.000061
Epoch: 0, Step: 1600, Average Loss: 0.000004
Epoch: 0, Step: 1700, Average Loss: 0.000000
Epoch: 0, Step: 1800, Average Loss: 0.000002
Epoch: 0, Step: 1900, Average Loss: 0.000004
Epoch: 0, Step: 2000, Average Loss: 0.000913
Epoch: 0, Step: 2100, Average Loss: 0.000302
Epoch: 0, Step: 2200, Average Loss: 0.000001
Epoch: 0, Step: 230

Epoch: 5, Step: 1900, Average Loss: 0.000000
Epoch: 5, Step: 2000, Average Loss: 0.000000
Epoch: 5, Step: 2100, Average Loss: 0.000002
Epoch: 5, Step: 2200, Average Loss: 0.000189
Epoch: 5, Step: 2300, Average Loss: 0.000001
Epoch: 5, Step: 2400, Average Loss: 0.000001
Epoch: 5, Step: 2500, Average Loss: 0.000001
Epoch: 5, Step: 2600, Average Loss: 0.000003
Epoch: 5, Step: 2700, Average Loss: 0.000000
Epoch: 5, Step: 2800, Average Loss: 0.000281
Epoch: 5, Step: 2900, Average Loss: 0.000000
Epoch: 5, Step: 3000, Average Loss: 0.000009
Epoch: 5, Step: 3100, Average Loss: 0.000003
Epoch: 5, Step: 3200, Average Loss: 0.000011
Epoch: 5, Step: 3300, Average Loss: 0.000024
Epoch: 5, Validation Loss: 0.001287
Epoch: 6, Step: 100, Average Loss: 0.000001
Epoch: 6, Step: 200, Average Loss: 0.000000
Epoch: 6, Step: 300, Average Loss: 0.000001
Epoch: 6, Step: 400, Average Loss: 0.000000
Epoch: 6, Step: 500, Average Loss: 0.000003
Epoch: 6, Step: 600, Average Loss: 0.000000
Epoch: 6, Step: 700, Aver

Epoch: 11, Step: 200, Average Loss: 0.000000
Epoch: 11, Step: 300, Average Loss: 0.000000
Epoch: 11, Step: 400, Average Loss: 0.000002
Epoch: 11, Step: 500, Average Loss: 0.000000
Epoch: 11, Step: 600, Average Loss: 0.000000
Epoch: 11, Step: 700, Average Loss: 0.000002
Epoch: 11, Step: 800, Average Loss: 0.000000
Epoch: 11, Step: 900, Average Loss: 0.000000
Epoch: 11, Step: 1000, Average Loss: 0.000006
Epoch: 11, Step: 1100, Average Loss: 0.000000
Epoch: 11, Step: 1200, Average Loss: 0.000000
Epoch: 11, Step: 1300, Average Loss: 0.000014
Epoch: 11, Step: 1400, Average Loss: 0.000000
Epoch: 11, Step: 1500, Average Loss: 0.000000
Epoch: 11, Step: 1600, Average Loss: 0.000000
Epoch: 11, Step: 1700, Average Loss: 0.000000
Epoch: 11, Step: 1800, Average Loss: 0.000000
Epoch: 11, Step: 1900, Average Loss: 0.000002
Epoch: 11, Step: 2000, Average Loss: 0.000000
Epoch: 11, Step: 2100, Average Loss: 0.000000
Epoch: 11, Step: 2200, Average Loss: 0.000000
Epoch: 11, Step: 2300, Average Loss: 0.000

Epoch: 16, Step: 1600, Average Loss: 0.000000
Epoch: 16, Step: 1700, Average Loss: 0.000000
Epoch: 16, Step: 1800, Average Loss: 0.000003
Epoch: 16, Step: 1900, Average Loss: 0.000000
Epoch: 16, Step: 2000, Average Loss: 0.000002
Epoch: 16, Step: 2100, Average Loss: 0.000097
Epoch: 16, Step: 2200, Average Loss: 0.000012
Epoch: 16, Step: 2300, Average Loss: 0.000001
Epoch: 16, Step: 2400, Average Loss: 0.000000
Epoch: 16, Step: 2500, Average Loss: 0.000000
Epoch: 16, Step: 2600, Average Loss: 0.000000
Epoch: 16, Step: 2700, Average Loss: 0.000001
Epoch: 16, Step: 2800, Average Loss: 0.000000
Epoch: 16, Step: 2900, Average Loss: 0.000001
Epoch: 16, Step: 3000, Average Loss: 0.000000
Epoch: 16, Step: 3100, Average Loss: 0.000000
Epoch: 16, Step: 3200, Average Loss: 0.000000
Epoch: 16, Step: 3300, Average Loss: 0.000001
Epoch: 17, Step: 100, Average Loss: 0.000001
Epoch: 17, Step: 200, Average Loss: 0.000000
Epoch: 17, Step: 300, Average Loss: 0.000000
Epoch: 17, Step: 400, Average Loss: 0

...

Epoch: 27, Step: 1100, Average Loss: 0.000000
Epoch: 27, Step: 1200, Average Loss: 0.000000
Epoch: 27, Step: 1300, Average Loss: 0.000001
Epoch: 27, Step: 1400, Average Loss: 0.000000
Epoch: 27, Step: 1500, Average Loss: 0.000000
Epoch: 27, Step: 1600, Average Loss: 0.000000
Epoch: 27, Step: 1700, Average Loss: 0.000001
Epoch: 27, Step: 1800, Average Loss: 0.000000
Epoch: 27, Step: 1900, Average Loss: 0.000000
Epoch: 27, Step: 2000, Average Loss: 0.000000
Epoch: 27, Step: 2100, Average Loss: 0.000000
Epoch: 27, Step: 2200, Average Loss: 0.000000
Epoch: 27, Step: 2300, Average Loss: 0.000000
Epoch: 27, Step: 2400, Average Loss: 0.000000
Epoch: 27, Step: 2500, Average Loss: 0.000002
Epoch: 27, Step: 2600, Average Loss: 0.000000
Epoch: 27, Step: 2700, Average Loss: 0.000000
Epoch: 27, Step: 2800, Average Loss: 0.000000
Epoch: 27, Step: 2900, Average Loss: 0.000000
Epoch: 27, Step: 3000, Average Loss: 0.000002
Epoch: 27, Step: 3100, Average Loss: 0.000117
Epoch: 27, Step: 3200, Average Los

Epoch: 32, Step: 2500, Average Loss: 0.000000
Epoch: 32, Step: 2600, Average Loss: 0.000000
Epoch: 32, Step: 2700, Average Loss: 0.000000
Epoch: 32, Step: 2800, Average Loss: 0.000000
Epoch: 32, Step: 2900, Average Loss: 0.000000
Epoch: 32, Step: 3000, Average Loss: 0.000000
Epoch: 32, Step: 3100, Average Loss: 0.000000
Epoch: 32, Step: 3200, Average Loss: 0.000001
Epoch: 32, Step: 3300, Average Loss: 0.000000
Epoch: 33, Step: 100, Average Loss: 0.000000
Epoch: 33, Step: 200, Average Loss: 0.000000
Epoch: 33, Step: 300, Average Loss: 0.000000
Epoch: 33, Step: 400, Average Loss: 0.000033
Epoch: 33, Step: 500, Average Loss: 0.000000
Epoch: 33, Step: 600, Average Loss: 0.000000
Epoch: 33, Step: 700, Average Loss: 0.000000
Epoch: 33, Step: 800, Average Loss: 0.000000
Epoch: 33, Step: 900, Average Loss: 0.000000
Epoch: 33, Step: 1000, Average Loss: 0.000000
Epoch: 33, Step: 1100, Average Loss: 0.000000
Epoch: 33, Step: 1200, Average Loss: 0.000000
Epoch: 33, Step: 1300, Average Loss: 0.0000

Epoch: 38, Step: 600, Average Loss: 0.000004
Epoch: 38, Step: 700, Average Loss: 0.000003
Epoch: 38, Step: 800, Average Loss: 0.000000
Epoch: 38, Step: 900, Average Loss: 0.000000
Epoch: 38, Step: 1000, Average Loss: 0.000000
Epoch: 38, Step: 1100, Average Loss: 0.000000
Epoch: 38, Step: 1200, Average Loss: 0.000000
Epoch: 38, Step: 1300, Average Loss: 0.000000
Epoch: 38, Step: 1400, Average Loss: 0.000000
Epoch: 38, Step: 1500, Average Loss: 0.000000
Epoch: 38, Step: 1600, Average Loss: 0.000000
Epoch: 38, Step: 1700, Average Loss: 0.000001
Epoch: 38, Step: 1800, Average Loss: 0.000000
Epoch: 38, Step: 1900, Average Loss: 0.000000
Epoch: 38, Step: 2000, Average Loss: 0.000000
Epoch: 38, Step: 2100, Average Loss: 0.000004
Epoch: 38, Step: 2200, Average Loss: 0.000000
Epoch: 38, Step: 2300, Average Loss: 0.000000
Epoch: 38, Step: 2400, Average Loss: 0.000001
Epoch: 38, Step: 2500, Average Loss: 0.000000
Epoch: 38, Step: 2600, Average Loss: 0.000000
Epoch: 38, Step: 2700, Average Loss: 0

Epoch: 43, Step: 2000, Average Loss: 0.000000
Epoch: 43, Step: 2100, Average Loss: 0.000000
Epoch: 43, Step: 2200, Average Loss: 0.000004
Epoch: 43, Step: 2300, Average Loss: 0.000000
Epoch: 43, Step: 2400, Average Loss: 0.000002
Epoch: 43, Step: 2500, Average Loss: 0.000008
Epoch: 43, Step: 2600, Average Loss: 0.000000
Epoch: 43, Step: 2700, Average Loss: 0.000000
Epoch: 43, Step: 2800, Average Loss: 0.000000
Epoch: 43, Step: 2900, Average Loss: 0.000000
Epoch: 43, Step: 3000, Average Loss: 0.000000
Epoch: 43, Step: 3100, Average Loss: 0.000000
Epoch: 43, Step: 3200, Average Loss: 0.000000
Epoch: 43, Step: 3300, Average Loss: 0.000001
Epoch: 44, Step: 100, Average Loss: 0.000000
Epoch: 44, Step: 200, Average Loss: 0.000000
Epoch: 44, Step: 300, Average Loss: 0.000000
Epoch: 44, Step: 400, Average Loss: 0.000000
Epoch: 44, Step: 500, Average Loss: 0.000000
Epoch: 44, Step: 600, Average Loss: 0.000001
Epoch: 44, Step: 700, Average Loss: 0.000000
Epoch: 44, Step: 800, Average Loss: 0.000

Epoch: 49, Step: 100, Average Loss: 0.000000
Epoch: 49, Step: 200, Average Loss: 0.000000
Epoch: 49, Step: 300, Average Loss: 0.000000
Epoch: 49, Step: 400, Average Loss: 0.000000
Epoch: 49, Step: 500, Average Loss: 0.000000
Epoch: 49, Step: 600, Average Loss: 0.000000
Epoch: 49, Step: 700, Average Loss: 0.000000
Epoch: 49, Step: 800, Average Loss: 0.000000
Epoch: 49, Step: 900, Average Loss: 0.000001
Epoch: 49, Step: 1000, Average Loss: 0.000000
Epoch: 49, Step: 1100, Average Loss: 0.000000
Epoch: 49, Step: 1200, Average Loss: 0.000000
Epoch: 49, Step: 1300, Average Loss: 0.000001
Epoch: 49, Step: 1400, Average Loss: 0.000000
Epoch: 49, Step: 1500, Average Loss: 0.000024
Epoch: 49, Step: 1600, Average Loss: 0.000000
Epoch: 49, Step: 1700, Average Loss: 0.000000
Epoch: 49, Step: 1800, Average Loss: 0.000010
Epoch: 49, Step: 1900, Average Loss: 0.000000
Epoch: 49, Step: 2000, Average Loss: 0.000000
Epoch: 49, Step: 2100, Average Loss: 0.000000
Epoch: 49, Step: 2200, Average Loss: 0.0000